# Análisis de vuelos

## 1. Importamos librerías y se crea el contexto

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType

In [3]:
spark = SparkSession.builder.appName("flights").config("inferSchema", "true").getOrCreate()

## 2. Leemos el fichero infiriendo el equema

In [4]:
flightsDF = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .csv("/data/flights_jan08.csv")

flightsDF.cache()

DataFrame[Year: int, Month: int, DayofMonth: int, DayOfWeek: int, DepTime: string, CRSDepTime: int, ArrTime: string, CRSArrTime: int, UniqueCarrier: string, FlightNum: int, TailNum: string, ActualElapsedTime: string, CRSElapsedTime: int, AirTime: string, ArrDelay: string, DepDelay: string, Origin: string, Dest: string, Distance: int, TaxiIn: string, TaxiOut: string, Cancelled: int, CancellationCode: string, Diverted: int, CarrierDelay: string, WeatherDelay: string, NASDelay: string, SecurityDelay: string, LateAircraftDelay: string]

## 3. Análisis de las columnas
### A. Nombres y tipos de datos

In [5]:
flightsDF.printSchema()
nfilas = flightsDF.count()
print("El DataFrame tiene", nfilas, "filas")

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Ca

### B. Interpretación de las columnas

En un vistazo podemos

* **Relacionadas con fechas:** *Year*, *Month*, *DayofMonth*, *DayOfWeek*, *DepTime*, *CRSDepTime*, *ArrTime* and *CRSArrTime*
* **Relacionadas con el vuelo:** *UniqueCarrier*, *FlightNum*, *TailNum*, *ActualElapsedTime*, *CRSElapsedTime*, *AirTime*, *Origin*, *Dest*, *Distance*, *TaxiIn* and *TaxiOut*
* **Relacionadas con problemas del vuelo:** *ArrDelay*, *DepDelay*, *Cancelled*, *CancellationCode*, *Diverted*, *CarrierDelay*, *WeatherDelay*, *NASDelay*, *SecurityDelay* and *LateAircraftDelay*

## 4. Exploración de las columnas relacionadas con fechas

In [6]:
print ("Resumen de las columnas Year, Month, DayofMonth and DayOfWeek:")
tempDF = flightsDF.select("Year","Month","DayofMonth","DayOfWeek")
summaryDF = tempDF.summary()
summaryDF.show()
summaryDF.count()

Resumen de las columnas Year, Month, DayofMonth and DayOfWeek:
+-------+------+-----+------------------+------------------+
|summary|  Year|Month|        DayofMonth|         DayOfWeek|
+-------+------+-----+------------------+------------------+
|  count| 86357|86357|             86357|             86357|
|   mean|2008.0|  1.0|15.973968526002524| 3.942378730155054|
| stddev|   0.0|  0.0| 7.640382177727954|2.0222466814572404|
|    min|  2008|    1|                 3|                 1|
|    25%|  2008|    1|                 9|                 2|
|    50%|  2008|    1|                16|                 4|
|    75%|  2008|    1|                23|                 6|
|    max|  2008|    1|                29|                 7|
+-------+------+-----+------------------+------------------+



8

In [18]:
print ("Resumen de las columnas Year, Month, DayofMonth and DayOfWeek:")
tempDF = flightsDF.select("Year","Month","DayofMonth","DayOfWeek")
summaryDF = tempDF.summary()
summaryDF.show()

print("Comprobamos valores nulos en Year, Month, DayofMonth and DayOfWeek:")
flightsDF.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in ["Year","Month","DayofMonth","DayOfWeek"]]).show()

print("Contamos los valores distintos que hay en las columnas Year, Month, DayofMonth and DayOfWeek:")
flightsDF.select(
    [F.countDistinct(c).alias(c) for c in ["Year","Month","DayofMonth","DayOfWeek"]]
).show()

print ("Ocurrencias más frecuentes y menos frecuentes en DayofMonth y DayOfWeek:")
dayofMonthOccurrencesDF = flightsDF.groupBy("DayofMonth").agg(F.count(F.lit(1)).alias("Total")).show()
dayOfWeekDF = flightsDF.groupBy("DayOfWeek").agg(F.count('*').alias("Total")).show()


Resumen de las columnas Year, Month, DayofMonth and DayOfWeek:
+-------+------+-----+------------------+------------------+
|summary|  Year|Month|        DayofMonth|         DayOfWeek|
+-------+------+-----+------------------+------------------+
|  count| 86357|86357|             86357|             86357|
|   mean|2008.0|  1.0|15.973968526002524| 3.942378730155054|
| stddev|   0.0|  0.0| 7.640382177727954|2.0222466814572404|
|    min|  2008|    1|                 3|                 1|
|    25%|  2008|    1|                 9|                 2|
|    50%|  2008|    1|                16|                 4|
|    75%|  2008|    1|                23|                 6|
|    max|  2008|    1|                29|                 7|
+-------+------+-----+------------------+------------------+

Comprobamos valores nulos en Year, Month, DayofMonth and DayOfWeek:
+----+-----+----------+---------+
|Year|Month|DayofMonth|DayOfWeek|
+----+-----+----------+---------+
|   0|    0|         0|        0|


In [8]:
leastFreqDayOfMonth    = dayofMonthOccurrencesDF.orderBy(F.col("Total").asc()).first()
mostFreqDayOfMonth     = dayofMonthOccurrencesDF.orderBy(F.col("Total").desc()).first()
leastFreqDayOfWeek     = dayOfWeekDF.orderBy(F.col("Total").asc()).first()
mostFreqDayOfWeek      = dayOfWeekDF.orderBy(F.col("Total").desc()).first()

print("Dia del mes más frecuente: {0} ({1} ocurrencias)".format(mostFreqDayOfMonth["DayofMonth"], mostFreqDayOfMonth["Total"]))
print("Dia del mes menos frecuente: {0} ({1} ocurrencias)".format(leastFreqDayOfMonth["DayofMonth"], leastFreqDayOfMonth["Total"]))
print("Dia de la semana más frecuente: {0} ({1} ocurrencias)".format(mostFreqDayOfWeek["DayOfWeek"], mostFreqDayOfWeek["Total"]))
print("Dia de la semana menos frecuente: {0} ({1} ocurrencias)".format(leastFreqDayOfWeek["DayOfWeek"], leastFreqDayOfWeek["Total"]))

print ("Resumen de las columnas DepTime, CRSDepTime, ArrTime and CRSArrTime:")
flightsDF.select("DepTime","CRSDepTime","ArrTime","CRSArrTime").summary().show()

print("Comprobamos los nulos en las columnas DepTime, CRSDepTime, ArrTime and CRSArrTime:")
flightsDF.select(
    [F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in ["DepTime","CRSDepTime","ArrTime","CRSArrTime"]]
).show()

print("Contamos los valores distintos que hay en las columnas DepTime, CRSDepTime, ArrTime and CRSArrTime:")
flightsDF.select(
    [F.countDistinct(c).alias(c) for c in ["DepTime","CRSDepTime","ArrTime","CRSArrTime"]]
).show()

Dia del mes más frecuente: 4 (3438 ocurrencias)
Dia del mes menos frecuente: 3 (2367 ocurrencias)
Dia de la semana más frecuente: 5 (13736 ocurrencias)
Dia de la semana menos frecuente: 3 (10292 ocurrencias)
Resumen de las columnas DepTime, CRSDepTime, ArrTime and CRSArrTime:
+-------+------------------+------------------+------------------+------------------+
|summary|           DepTime|        CRSDepTime|           ArrTime|        CRSArrTime|
+-------+------------------+------------------+------------------+------------------+
|  count|             86357|             86357|             86357|             86357|
|   mean|1359.5908617318926|1346.0954525979364|1495.0254847255762| 1503.791122896812|
| stddev| 464.4391438174021|452.63563353626046|496.69551687915265|479.62566806413247|
|    min|               100|               600|                 1|                 5|
|    25%|             946.0|               945|            1115.0|              1120|
|    50%|            1346.0|       

## 5. Respondemos a algunas preguntas de negocio para ver si podemos mejorar el servicio

### A. Porcentaje de vuelos retrasados y no cancelados por severidad

In [9]:
# Quitamos el string "NA", que no es ningún valor especial para Spark, solo un string más
delayCategorizationDF = flightsDF.where(F.col("ArrDelay") != "NA")

Vamos a categorizar el retraso en la llegada como sigue:

  "nodelay"      - delay=(-infinito,0] mins

  "acceptable"   - delay=(0,15] mins

  "annoying"     - delay=(15,30] mins

  "impactul"     - delay=(30,60] mins
  
  "unacceptable" - delay=(60,+infinito) mins

In [20]:
# 1. Añadimos nuestra categorización
totalFlights = flightsDF.count()
delayCategorizationDF = flightsDF\
   .where(F.col("ArrDelay")!="NA")\
   .withColumn("DelaySeverity", F.when(F.col("ArrDelay")<=0,"1.nodelay")\
                               .when((F.col("ArrDelay")>0) & (F.col("ArrDelay")<=15),"2.acceptable")\
                               .when((F.col("ArrDelay")>15) & (F.col("ArrDelay")<=30),"3.annoying")\
                               .when((F.col("ArrDelay")>30) & (F.col("ArrDelay")<=60),"4.impactful")\
                               .otherwise("5.unacceptable"))

delayCategorizationDF.cache() # Esto optimiza la velocidad

# 2. Calculamos el porcentaje de vuelos que caen en cada categoría de retraso
delayCategorizationDF.where(F.col("Cancelled")==0)\
                     .select("DelaySeverity")\
                     .groupBy("DelaySeverity")\
                     .agg(F.count("DelaySeverity").alias("NumFlights"), \
                          (F.count("DelaySeverity")/totalFlights*100).alias("Ratio"))\
                     .orderBy("DelaySeverity")\
                     .select("DelaySeverity","NumFlights",F.round("Ratio",2).alias("RoundedRatio")).show()

+--------------+----------+------------+
| DelaySeverity|NumFlights|RoundedRatio|
+--------------+----------+------------+
|     1.nodelay|     49430|       57.24|
|  2.acceptable|     19954|       23.11|
|    3.annoying|      7075|        8.19|
|   4.impactful|      5202|        6.02|
|5.unacceptable|      3645|        4.22|
+--------------+----------+------------+



### B. Estadísticas sobre el tipo de retraso para los vuelos que han sufrido retraso severo (carrier, weather, NAS, security and lateaircraft)

Para calcular estadísticas de los vuelos con retraso severo, tenemos que preparar el DataFrame previo (delayCategorizationDF):

  1. Quitar vuelos cancelados

  2. Quitar vuelos cuyo retraso sea "no delay" o "acceptable"

  3. Convertir columnas numéricas a enteros
  
     (no hay "NA" en el DF resultante severeDelaysDF)

In [11]:
severeDelaysDF = \
  delayCategorizationDF.where((F.col("Cancelled")==0))\
                       .where((F.col("DelaySeverity")!="1.nodelay") & (F.col("DelaySeverity")!="2.acceptable"))\
                       .withColumn("IntArrDelay", F.col("ArrDelay").cast(IntegerType()))\
                       .withColumn("IntCarrierDelay", F.col("CarrierDelay").cast(IntegerType()))\
                       .withColumn("IntWeatherDelay", F.col("WeatherDelay").cast(IntegerType()))\
                       .withColumn("IntNASDelay", F.col("NASDelay").cast(IntegerType()))\
                       .withColumn("IntSecurityDelay", F.col("SecurityDelay").cast(IntegerType()))\
                       .withColumn("IntLateAircraftDelay", F.col("LateAircraftDelay").cast(IntegerType()))\
                       .select("DelaySeverity", "IntArrDelay","IntCarrierDelay","IntWeatherDelay",\
                               "IntNASDelay", "IntSecurityDelay", "IntLateAircraftDelay")

severeDelaysDF.cache() # optimization to make the processing faster

print("Estadísticas sobre el retraso de los vuelos (en minutos):")
severeDelaysDF.groupBy("DelaySeverity")\
              .agg(F.avg("IntArrDelay").alias("AverageDelay"),\
                   F.min("IntArrDelay").alias("LowestDelay"),\
                   F.max("IntArrDelay").alias("HighestDelay"),\
                   F.stddev("IntArrDelay").alias("StdDevDelay"))\
              .orderBy("DelaySeverity").show()

print("Estadísticas sobre el retraso debido al avión")
severeDelaysDF.groupBy("DelaySeverity")\
              .agg(F.avg("IntCarrierDelay").alias("AverageDelay"),\
                   F.min("IntCarrierDelay").alias("LowestDelay"),\
                   F.max("IntCarrierDelay").alias("HighestDelay"),\
                   F.stddev("IntCarrierDelay").alias("StdDevDelay"))\
              .orderBy("DelaySeverity").show()

print("Estadísticas sobre el retraso debido al tiempo atmosférico")
severeDelaysDF.groupBy("DelaySeverity")\
              .agg(F.avg("IntWeatherDelay").alias("AverageDelay"),\
                   F.min("IntWeatherDelay").alias("LowestDelay"),\
                   F.max("IntWeatherDelay").alias("HighestDelay"),\
                   F.stddev("IntWeatherDelay").alias("StdDevDelay"))\
              .orderBy("DelaySeverity").show()

Estadísticas sobre el retraso de los vuelos (en minutos):
+--------------+------------------+-----------+------------+------------------+
| DelaySeverity|      AverageDelay|LowestDelay|HighestDelay|       StdDevDelay|
+--------------+------------------+-----------+------------+------------------+
|    3.annoying|21.987561837455832|         16|          30| 4.201271154113909|
|   4.impactful| 42.44444444444444|         31|          60| 8.439367702313486|
|5.unacceptable| 107.4554183813443|         61|         500|49.663544169822934|
+--------------+------------------+-----------+------------+------------------+

Estadísticas sobre el retraso debido al avión
+--------------+------------------+-----------+------------+------------------+
| DelaySeverity|      AverageDelay|LowestDelay|HighestDelay|       StdDevDelay|
+--------------+------------------+-----------+------------+------------------+
|    3.annoying| 5.352932862190813|          0|          30| 7.442700984333453|
|   4.impactful

### C. Top 20 aeropuertos involucrados en retrasos severos

Vamos a dar como respuesta:
   1. Lista de los 20 aeropuertos de origen con el porcentaje más alto de vuelos con retraso severo (sobre el total de vuelos)
   2. Lista de los 20 aeropuertos de origen con el mayor porcentaje de vuelos en cada categoría de retraso (unacceptable,
      impactful y annoying)

Para ello:
   1. Creamos un DataFrame con el número total de vuelos por aeropuerto de origen, que actuarán como denominadores (totalFlightsOriginDF)
   2. Creamos otro DataFrame con datos agregados por aeropuerto de origen y severidadpara contar el número de vuelos retrasados por categoría de severidad (severeDelaysOriginDF)
   3. Combinamos ambos DataFrames para calcular otro donde dividimos cada conteo entre el total de vuelos que salen de ese aeropuerto

In [24]:
totalFlightsOriginDF = \
   flightsDF.groupBy("Origin")\
            .agg(F.count("ArrDelay").alias("TotalFlights"))

severeDelaysOriginDF = \
  delayCategorizationDF.where((F.col("Cancelled")==0))\
                       .where((F.col("DelaySeverity")!="1.nodelay") & (F.col("DelaySeverity")!="2.acceptable"))\
                       .withColumn("IntArrDelay", F.col("ArrDelay").cast(IntegerType()))\
                       .select("DelaySeverity", "IntArrDelay","Origin")\
                       .groupBy("Origin", "DelaySeverity")\
                       .agg(F.count("IntArrDelay").alias("NumSevereDelayedFlights"))
combinedDF = \
  severeDelaysOriginDF\
     .join(totalFlightsOriginDF, "Origin")\
     .withColumn("SevereDelayedRatio", F.round(F.col("NumSevereDelayedFlights")/F.col("TotalFlights")*100,2))\
     .orderBy(F.col("SevereDelayedRatio").desc())

combinedDF.cache() # Optimización para hacer lo próximo más rápido

print("Top 20 aeropuertos de origen con el porcentaje más alto de vuelos con retraso severo:")
combinedDF.limit(20).show()

print("Ídem, por categoría de severidad")
combinedDF\
   .groupBy("Origin")\
   .pivot("DelaySeverity")\
   .agg(F.min("SevereDelayedRatio"))\
   .orderBy(F.col("`5.unacceptable`").desc(), F.col("`4.impactful`").desc(), F.col("`3.annoying`").desc())\
   .limit(20).show()



Top 20 aeropuertos de origen con el porcentaje más alto de vuelos con retraso severo:
+------+--------------+-----------------------+------------+------------------+
|Origin| DelaySeverity|NumSevereDelayedFlights|TotalFlights|SevereDelayedRatio|
+------+--------------+-----------------------+------------+------------------+
|   SFO|5.unacceptable|                    136|         665|             20.45|
|   GEG|    3.annoying|                     48|         401|             11.97|
|   LAS|    3.annoying|                    673|        6302|             10.68|
|   OMA|    3.annoying|                     41|         395|             10.38|
|   ONT|    3.annoying|                    146|        1424|             10.25|
|   RNO|    3.annoying|                    101|         998|             10.12|
|   SLC|    3.annoying|                    123|        1215|             10.12|
|   LAX|    3.annoying|                    314|        3121|             10.06|
|   SLC|5.unacceptable|           